In [0]:
import glob
from PIL import Image
from numpy import expand_dims
import os

이미지 경로들을 불러와, 파일 이름에 포함되어 있는 번호를 기준으로 자기가 배정받은 번호만 my_img_pathes에 저장.
category_nums는 파일 이름에 포함된 번호들을 따로 모아놓는 리스트.(번호 = 카테고리 이므로 하나의 카테고리가 몇 장 들어있는 지 세기 위함)

In [0]:
img_pathes = glob.glob("/content/drive/Shared drives/선빵팀 :)/TuningStar_최종/TuningStar/*.jpg") # 경로를 필요한 이미지들이 저장되어 있는 경로로 지정.
my_img_pathes=[]
category_nums=[]
for img_path in img_pathes:
  num_jpg = img_path.split("/")[-1]
  category_num = num_jpg.split('.')[0]
  try:
    category_num = int(category_num)
  except:
    if "(" in category_num:
      category_num = int(category_num.split("(")[0])
    elif "-" in category_num:
      category_num = int(category_num.split('-')[0])
  if category_num >= 302 and category_num <= 452: # 비교하는 번호를 자기가 배정받은 번호로 수정.
    my_img_pathes.append(img_path)
    category_nums.append(category_num)

Keras의 ImageDataGenerator를 통해 Data Augmentation을 할 수 있는 generator생성.

In [0]:
## data augmentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

##ImageDataGenerator에 원하는 옵션들을 추가해주면 옵션들 적용하면서 랜덤하게 생성해줍니다.
datagen = ImageDataGenerator(
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        brightness_range=[0.2, 1.0],
        vertical_flip=True,
        horizontal_flip=True,
        fill_mode= 'nearest')

각 카테고리 번호 별로 몇 개씩 들어있는 지 카운팅한 뒤, 카테고리(key):등장횟수(value) 형태로 딕셔너리 생성

In [0]:
category_count_dict = {}
for i in range(302,453): # 번호 범위를 자기가 배정받은 번호로 수정.
  num_count = category_nums.count(i)
  category_count_dict[i] = num_count

배정받은 이미지 경로들을 통해 이미지를 가져오고,
하나의 카테고리를 하나의 폴더로 만듦. 위에서도 이야기했듯이 파일 번호 = 카테고리이므로 이미 카테고리 폴더가 생성되어있는 경우는 넘어가게 햇음.

In [0]:
for img_path in my_img_pathes:
  num_jpg = img_path.split("/")[-1]
  category_num = num_jpg.split('.')[0]
  img_name = category_num
  try:
    category_num = int(category_num)
  except:
    if "(" in category_num:
      category_num = int(category_num.split("(")[0])
    elif "-" in category_num:
      category_num = int(category_num.split('-')[0])
  category_count = category_count_dict[category_num]

  img = load_img(img_path)
  x = img_to_array(img)
  x = expand_dims(x,0)

  folder_path = "/content/drive/Shared drives/선빵팀 :)/Augmented_wheel/"+str(category_num) # folder_path는 자기가 저장할 경로에 맞게 수정해주어야 함. str(category)는 건들지 말고 앞부분만 수정.
  
  if not(os.path.isdir(folder_path)):
    os.makedirs(folder_path)
    
  img_num = int(10000/category_count)  # 총 10000장을 Augmentation할 건데, 카테고리 카운트가 1보다 큰 이미지는, my_img_pathes안에 같은 카테고리에 속하는 이미지가 또 있다는 의미.
                                       # 만약 1번카테고리에 해당하는 이미지가 2장이 있다면, 각각 5000장씩 Augmentation해서 총 1000장을 맞춰주기 위해 categroy_count로 나누어줌.

  if 10000%category_count != 0: # 여기서 print의 결과로 나오는 이미지들은 10000의 약수가 아니므로, augmentation한 결과가 정확히 10000장이 되지 못함. 해당 카테고리에서는 수동으로 이미지 복붙해서 10000장 맞춰주세요.
    print(category_num)
  
  i=0
  for batch in datagen.flow(x, batch_size=32, save_to_dir=folder_path, save_prefix=img_name, save_format="jpeg"):
    i+=1
    if i > img_num:
      break

321
321
321
321
321


KeyboardInterrupt: ignored